In [ ]:
# Importing Relevant Libraries
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import time

In [ ]:
# Load the pre-trained model
model = load_model('carrot_dummy_model.h5')

# Print model summary to verify
model.summary()

In [ ]:
def preprocess_frame(frame):
    #Preprocess the frame to fit the model's input requirements
    # Resize the frame to the size expected by the model
    frame_resized = cv2.resize(frame, (512, 512))
    # Normalize the pixel values to be between 0 and 1
    frame_normalized = frame_resized / 255.0
    # Expand dimensions to match model input shape (1, 512, 512, 3)
    frame_expanded = np.expand_dims(frame_normalized, axis=0)
    return frame_expanded

def classify_frame(model, frame):
    #Classify the frame and return the prediction
    processed_frame = preprocess_frame(frame)
    prediction = model.predict(processed_frame)
    return prediction

In [ ]:
# Initialize video capture (0 for webcam, or provide a video file path)
video_capture = cv2.VideoCapture(0)

# Check if the video capture is opened
if not video_capture.isOpened():
    print("Error: Could not open video stream.")
else:
    print("Video stream opened successfully.")

# Set up variables to keep track of detected carrots
carrot_count = 0
recorded_frames = []

# Define threshold for carrot detection (example threshold is used)
carrot_threshold = 0.5

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    if not ret:
        print("Failed to grab frame.")
        break

    # Classify the frame
    prediction = classify_frame(model, frame)
    
    # Assuming the model outputs a single value indicating carrot likelihood
    if prediction[0][0] > carrot_threshold:
        carrot_count += 1
        recorded_frames.append(frame)
        label = "Carrot: {:.2f}%".format(prediction[0][0] * 100)
    else:
        label = "Not Carrot: {:.2f}%".format((1 - prediction[0][0]) * 100)

    # Display the label on the frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture when done
video_capture.release()
cv2.destroyAllWindows()

print(f"Total carrots detected: {carrot_count}")

In [ ]:
# Save recorded frames to drive
for i, frame in enumerate(recorded_frames):
    filename = f"carrot_frame_{i}.jpg"
    cv2.imwrite(filename, frame)

print("Frames have been saved.")